# Step 1 : Installing the required libraries

In [1]:
!pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]

  Using cached numpy-1.18.5-cp38-cp38-win_amd64.whl (12.8 MB)


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\numpy\\LICENSE.txt'
Consider using the `--user` option or check the permissions.



  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Rolling back uninstall of numpy
  Moving to c:\users\win-10\appdata\roaming\python\python38\scripts\f2py.exe
   from C:\Users\win-10\AppData\Local\Temp\pip-uninstall-l9o_82yo\f2py.exe
  Moving to c:\users\win-10\appdata\roaming\python\python38\site-packages\numpy-1.19.5.dist-info\
   from C:\Users\win-10\AppData\Roaming\Python\Python38\site-packages\~umpy-1.19.5.dist-info
  Moving to c:\users\win-10\appdata\roaming\python\python38\site-packages\numpy\
   from C:\Users\win-10\AppData\Roaming\Python\Python38\site-packages\~~mpy


# 2. Testing gym environment 

In [1]:
import gym 
import random

In [2]:
pip install atari-py

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [ ]:
#1.run again from here if error occurs 

In [16]:
env = gym.make('Assault-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [17]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'UP', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [18]:
channels

3

In [19]:
#Random environment, DQN has not been applied so the AI plays randomly

In [20]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    for t in range(100):
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()



Episode:1 Score:63.0
Episode:2 Score:21.0
Episode:3 Score:21.0
Episode:4 Score:21.0
Episode:5 Score:21.0
Episode:6 Score:21.0
Episode:7 Score:42.0
Episode:8 Score:21.0
Episode:9 Score:63.0
Episode:10 Score:21.0


# 3. Creating a Deep Learning Model

In [21]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [22]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [23]:
model = build_model(height, width, channels, actions)

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 61, 39, 32)     6176      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 29, 18, 64)     32832     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 27, 16, 64)     36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 82944)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               42467840  
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_6 (Dense)              (None, 7)                

# 4. Building a DQN Agent with Keras

In [25]:
# if error occurs then run the code from beginning again from top

In [26]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [27]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [28]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

In [19]:
#2. if error occurs go to 1. and run each cell once again
#3. after executing the above code dqn.compile(). Then Run the line 4.

In [31]:
#testing for 10000 steps for ten-thousand steps, PS- this will take a while

In [32]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
  398/10000: episode: 1, duration: 15.690s, episode steps: 398, steps per second:  25, episode reward: 168.000, mean reward:  0.422 [ 0.000, 21.000], mean action: 2.794 [0.000, 6.000],  loss: --, mean_q: --, mean_eps: --
 1002/10000: episode: 2, duration: 42.629s, episode steps: 604, steps per second:  14, episode reward: 189.000, mean reward:  0.313 [ 0.000, 21.000], mean action: 2.808 [0.000, 6.000],  loss: 97.381104, mean_q: 11.451048, mean_eps: 0.909910
 1693/10000: episode: 3, duration: 663.334s, episode steps: 691, steps per second:   1, episode reward: 189.000, mean reward:  0.274 [ 0.000, 21.000], mean action: 3.048 [0.000, 6.000],  loss: 5.184674, mean_q: 12.687715, mean_eps: 0.878770
 2291/10000: episode: 4, duration: 569.832s, episode steps: 598, steps per second:   1, episode reward: 189.000, mean reward:  0.316 [ 0.000, 21.000], 

In [30]:
scores = dqn.test(env, nb_episodes=13, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 13 episodes ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Episode 1: reward: 126.000, steps: 784
Episode 2: reward: 147.000, steps: 832
Episode 3: reward: 168.000, steps: 802
Episode 4: reward: 189.000, steps: 944
Episode 5: reward: 189.000, steps: 1046
Episode 6: reward: 147.000, steps: 821
Episode 7: reward: 105.000, steps: 708
Episode 8: reward: 252.000, steps: 915
Episode 9: reward: 273.000, steps: 891
Episode 10: reward: 273.000, steps: 1148
Episode 11: reward: 84.000, steps: 1048
Episode 12: reward: 189.000, steps: 856
Episode 13: reward: 147.000, steps: 927
176.07692307692307


In [ ]:
#5. run above line to play after loading the weights 'dqn.test()'

# 5. Reload the weights to play the last game

In [ ]:
#run this for saving (already saved)

In [37]:
dqn.save_weights('dqn_weights.h5f')

[WARNING] dqn_weights.h5f.index already exists - overwrite? [y/n]y
[TIP] Next time specify overwrite=True!


In [ ]:
#4. run the below cell for loading already trained data then go to dqn.test cell to test it. and go to line 5 to test it

In [29]:
dqn.load_weights('dqn_weights.h5f')